<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://github.com/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Use this notebook to get alignment information from ImageJ image registration. Print stats for individual barcodes and calculate mapping from angled tissue slice to flat view.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
# @title Notebook Inputs { display-mode: "form" }
## @markdown How to use this notebook: [See Instructions]()
import numpy as np

# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
trakem_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/align3.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack.
# template_patch_1 = 11 # @param {type:"integer"}
# template_patch_2 = 14 # @param {type:"integer"}
# template_patch_3 = 17 # @param {type:"integer"}

# @markdown Enter template patch IDs in order, separated by commas. Example: 11, 14, 17
template_patch_list = "11, 14, 17" # @param {type:"string"}

# @markdown Z-depth of each template, in um. Enter as a comma-separated list. Example: 50, 52, 54
template_z_list = "68, 52, 56" # @param {type:"string"}

# @markdown Real (x,y) locations of photobleach barcode centers, as specified by script used to photobleach. Enter in format: (1000,0), (0, 1000), (0,0)
real_centers = [0,1000], [1000, 1000], [0, 0] # @param

real_centers = np.array(real_centers)
real_centers = np.column_stack((real_centers, np.zeros(real_centers.shape[0])))

template_size = 401
um_per_pixel = 2

from google.colab import drive
from google.colab import files
# drive.mount('/content/drive/')

assert len(template_patch_list) == len(template_z_list), "Number of elements in template patch list and template z list must match"

if not trakem_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  trakem_xml_path = list(uploaded.keys())[0]
  trakem_xml_path = os.path.join(os.getcwd(), trakem_xml_path)


In [3]:
# @title Environment Setup
!git clone https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

# from plane.fit_plane import FitPlane
from plane.fit_template import FitTemplate
# from plane.fit_plane import FitPlane
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

%cd ..

Cloning into 'coregister-xy'...
remote: Enumerating objects: 514, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 514 (delta 84), reused 76 (delta 57), pack-reused 396 (from 1)
Receiving objects: 100% (514/514), 1.04 MiB | 3.22 MiB/s, done.
Resolving deltas: 100% (299/299), done.
/content/coregister-xy
/content


In [49]:
import pandas as pd

# @title Compute Physical Transform Parameters, relative to image origin
ps = []
templates = [int(x) for x in template_patch_list.split(",")]
zs = [float(x) for x in template_z_list.split(",")]
num_templates = len(templates)

ps = []
for i in range(0,num_templates):
    project = FitTemplate.from_imagej_xml(trakem_xml_path, fluorescent_patch_number, templates[i], None, True)
    ps.append(project)

projects_data = {
    "Template ID": [i for i in range(1, num_templates+1)],
    "Patch Number": [t for t in templates],
    "Z (um)": [z for z in zs],
    "Center (x)": [project.tx + template_size/2 for project in ps],
    "Center (y)": [project.ty + template_size/2 for project in ps],
    "Rotation (deg)": [project.theta_deg for project in ps],
    "Scaling": [project.scale for project in ps],
    "Shear magnitude": [project.shear_magnitude for project in ps],
    "Shear vector (x)": [project.shear_vector[0] for project in ps],
    "Shear vector (y)": [project.shear_vector[1] for project in ps]
}

columns_to_summarize = ["Z (um)", "Rotation (deg)", "Scaling", "Shear magnitude", "Shear vector (x)", "Shear vector (y)"]

# Create DataFrame
df = pd.DataFrame(projects_data)

# Compute mean and standard deviation for selected columns only
mean_row = df[columns_to_summarize].mean()
std_row = df[columns_to_summarize].std()

# Append mean and std as new rows for selected columns only
summary_df = df.copy()
summary_df.loc['Mean', columns_to_summarize] = mean_row
summary_df.loc['StDev', columns_to_summarize] = std_row
summary_df = summary_df.round(2)
summary_df = summary_df.replace(np.nan, '', regex=True)

print("Stats for individual barcodes:\n")
print(summary_df)

# convert centers to distance, then append zs on axis
centers  = [(project.tx + template_size/2, project.ty + template_size/2) for project in ps]
centers = np.array(centers)
avgscale = np.mean([project.scale for project in ps])
centers = centers * (um_per_pixel / avgscale) # convert fluorescent units from pixels to um
centers_z = np.column_stack((centers, zs))

Stats for individual barcodes:

      Template ID Patch Number  Z (um) Center (x) Center (y)  Rotation (deg)  \
0             1.0         11.0   68.00     698.93    2920.88            7.28   
1             2.0         14.0   52.00    1658.59    3113.83            7.32   
2             3.0         17.0   56.00     738.31    1995.97            6.21   
Mean                             58.67                                  6.94   
StDev                             8.33                                  0.63   

       Scaling  Shear magnitude  Shear vector (x)  Shear vector (y)  
0         1.96            -0.05             -0.99             -0.10  
1         1.87             0.03              0.72              0.70  
2         1.92             0.03              0.79             -0.62  
Mean      1.92             0.00              0.17             -0.01  
StDev     0.04             0.05              1.01              0.66  


In [51]:
# UVH mapping
# for a point u,v:
# [x,y,z] = vec_u * u + vec_v * v + vec_h

u = np.array([x[0] for x in centers_z])
v = np.array([x[1] for x in centers_z])

x = np.array([x[0] for x in real_centers])
y = np.array([x[1] for x in real_centers])
z = np.ones_like(y)

# Number of points
n = u.shape[0]

A = np.zeros((3 * n, 9))
for i in range(n):
    A[3 * i] = [u[i], v[i], 1, 0, 0, 0, 0, 0, 0]      # x equation
    A[3 * i + 1] = [0, 0, 0, u[i], v[i], 1, 0, 0, 0]  # y equation
    A[3 * i + 2] = [0, 0, 0, 0, 0, 0, u[i], v[i], 1]  # z equation

# Output vector b
b = np.zeros(3 * n)
for i in range(n):
    b[3 * i] = x[i]
    b[3 * i + 1] = y[i]
    b[3 * i + 2] = z[i]

# Solve using least squares
M, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None) # TODO check that lol

ux, vx, hx, uy, vy, hy, uz, vz, hz = M

print("Transformation coefficients:")
print("Vector u:", [ux, uy, uz])
print("Vector v:", [vx, vy, vz])
print("Vector h:", [hx, hy, hz])

Transformation coefficients:
Vector u: [0.9898098141499, -0.20648898705143098, 7.828083447596932e-14]
Vector v: [0.04214533755918909, 1.0270001463964253, -3.112916410108851e-14]
Vector h: [-850.625131627435, -1980.5578118495184, 1.0000000000002434]


In [58]:
def project_onto_flat(u, v, h, points):
    u_coords = points[:, 0]
    v_coords = points[:, 1]

    # Project each (u, v) pair using the vectors u and v
    projected = np.outer(u_coords, u) + np.outer(v_coords, v) + h
    return projected

def compute_avg_error(a, b):
    assert a.shape == b.shape, "Input arrays must have the same shape"
    if a.shape[1] == 3:
        a = a[:, :2]
        b = b[:, :2]
    return np.mean(np.linalg.norm(a - b, axis=1))

u = np.array([ux, uy, uz])
v = np.array([vx, vy, vz])
h = np.array([hx, hy, hz])

projected = project_onto_flat(u,v,h, centers_z)
err = compute_avg_error(projected[:,:-1], real_centers[:,:-1])
assert round(err) == 0